# Linear discriminant analysis
Supervised data compression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import pandas as pd

#Load data
dataObj = load_wine()
X = dataObj.data
y = dataObj.target

# Create DataFrame with features
df = pd.DataFrame(X)
df.columns = dataObj.feature_names

# Add class column
df.insert(loc=0, column="Class", value=y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardize
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

## Method 1: Using NumPy

### Compute the "scaled" within-class scatter matrix
This is essentially covariance matrix


In [ ]:
# Number of features
d = X_train_std.shape[1]

# Scaled within-class scatter matrix
S_W = np.zeros((d, d))

for label in np.unique(y):
    S_i = np.cov(X_train_std[y_train == label], rowvar=False)
    S_W += S_i

df = pd.DataFrame(S_W)
display(df)

### Calculate the mean vectors $\mu_i$ calculated from sample from class $i$

In [ ]:
mvs = []

for label in np.unique(y):
    mv = np.mean(X_train_std[y_train == label, :], axis=0)
    mvs.append(mv)
    #mv_print = ", ".join(f"{i:5.2f}" for i in mv)
    #print(f"Label={label}, MV:{mv_print}")

df = pd.DataFrame(mvs)
df.index.name = 'Label'
display(df)

### Calculate the mean vector $\mu$ computed all from observations
(Actually this will give all zeros becuase data is already standardized.)

In [ ]:
mva = np.mean(X_train_std, axis=0).reshape(d,1) # make column vector

### Compute the between-class scatter matrix

In [ ]:
S_B = np.zeros((d, d))
for label, mv in zip(np.unique(y), mvs):
    # Count number of obervation in each class
    n = X_train[y_train == label, :].shape[0]
    # Mean values for each class
    mv = mv.reshape(d, 1)  # make column vector
    mmm = mv - mva
    # Outer product
    S_B += n * mmm @ mmm.T

df = pd.DataFrame(S_B)
display(df)


### Computer eigenvalues and eigenvectors
Solve the generalized eigenvalue problem for the matrix $S_W^{-1}S_B$:

In [ ]:
eigen_vals, eigen_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

# Print
display(pd.DataFrame(eigen_vals.real.reshape(1,-1)))
display(pd.DataFrame(eigen_vecs.real))


### Sort eigenvectors in decreasing order of the eigenvalues

In [ ]:
#Sort eigenvalues
idx = np.argsort(eigen_vals)
idx = idx[::-1] #Sort from max to min
eigen_vals = eigen_vals[idx]
eigen_vecs = eigen_vecs[:,idx]

# Print
display(pd.DataFrame(eigen_vals.real.reshape(1,-1)))
display(pd.DataFrame(eigen_vecs.real))

### Visualizing "discriminability"

In [ ]:
tot = sum(eigen_vals.real)
discr = eigen_vals.real/tot
cum_discr = np.cumsum(discr)

plt.bar(range(1, 14), discr, alpha=0.5, align='center', label='individual "discriminability"')
plt.step(range(1, 14), cum_discr, where='mid', label='cumulative "discriminability"')
plt.ylabel('"discriminability" ratio')
plt.xlabel('Linear Discriminants')
plt.ylim([-0.1, 1.1])
plt.legend(loc='best')
plt.tight_layout()
# plt.savefig('./figures/lda1.png', dpi=300)
plt.show()

### Transformation matrix, W

In [ ]:
w = eigen_vecs[:,[0,1]].real
display(pd.DataFrame(w))

### Projecting samples onto the new feature space

In [ ]:
X_train_lda = X_train_std.dot(w)

display(pd.DataFrame(X_train_lda))

### Visualizing training data

In [ ]:
from PlotFunction3 import plot_reduced_dim
plot_reduced_dim(X_train_lda, y_train, "LDA")

## Method 2: SKL

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA()
X_train_lda = lda.fit_transform(X_train_std, y_train)
lda.explained_variance_ratio_

In [ ]:
plt.bar(range(1, 3), lda.explained_variance_ratio_, alpha=0.5, align='center')
plt.step(range(1, 3), np.cumsum(lda.explained_variance_ratio_), where='mid')
plt.ylabel('"discriminability" ratio')
plt.xlabel('Linear Discriminants')
plt.show()

In [ ]:
lda = LDA(n_components=2)
X_train_lda = lda.fit_transform(X_train_std, y_train)
X_test_lda = lda.transform(X_test_std)

plot_reduced_dim(X_train_lda, y_train, "LDA")

## Training with logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr = lr.fit(X_train_lda, y_train)

In [ ]:
from PlotFunction2 import plot_decision_surface2

plot_decision_surface2(X_train_lda, X_test_lda, y_train, y_test, lr)